#  Multi-Objective Evolutionary Optimization

## Overview

In this notebook, we apply many-objective evolutionary optimization to explore **non-dominated policy sets** for dike reinforcement and spatial interventions within the Overijssel region. Our aim is to identify robust strategies that balance the interests of multiple stakeholders while respecting specific physical and institutional constraints.

## Scope of the Analysis

**Focus region**: The optimization is focused on two dike rings:
  - **Deventer** (`A.4`)
  - **Grossel** (`A.5`)


**Constraints applied**: No initial constratint is applied to the dike model to avoid premature aggregation. By doing so the optimization will allow to explore the trade-offs in the complete set of scenarios and levers


**Candidate policies**: a selection of candidate policies will be performed by applying constraints to the optimization results. Identified candidate policies will then be subject to robustness analysis.


**Decision space**:We search over policy levers, which include dike reinforcements and RfR implementations. All other interventions are treated as optional and will be explored by the algorithm.


**Evaluation criteria**: Objectives reflect the priorities of the *Province of Overijssel*, such as minimizing flood risk and economic damages, while preserving social and ecological outcomes.


## Purpose

This analysis uses the `optimize()` functionality from the `ema_workbench`, relying on an $\epsilon$-NSGAII algorithm to:
- Discover a set of **Pareto-optimal policies** under complex trade-offs.
- Support negotiation and coordination between regional actors (e.g., RWS, local municipalities, and the province).
- Identify **robust** strategies that perform well under a wide range of uncertain future scenarios.


## Key Features

- Use of $\epsilon$-archiving and **parallel coordinate plots** to visualize trade-offs.
- Inclusion of **convergence metrics** and optional seed analysis to assess solution quality.
- Integration of **realistic policy constraints** from stakeholders to select candidate policies.

In [1]:
# Standard library imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Third-party libraries
import seaborn as sns

# EMA Workbench core components
from ema_workbench import (
    ema_logging,
    load_results,
    MultiprocessingEvaluator,
    Scenario,
    HypervolumeMetric,
    GenerationalDistanceMetric,
    EpsilonIndicatorMetric,
    InvertedGenerationalDistanceMetric,
    IntegerParameter,
    SpacingMetric,
)

# EMA Workbench analysis and optimization utilities
from ema_workbench.analysis import parcoords
from ema_workbench.em_framework.optimization import (
    ArchiveLogger,
    EpsilonProgress,
    to_problem,
)

# Local project-specific modules
from problem_formulation import get_model_for_problem_formulation

In [2]:
ema_logging.log_to_stderr(level=ema_logging.INFO)

<Logger EMA (DEBUG)>

In [3]:
problem_formulation_id = 3

dike_model, _ = get_model_for_problem_formulation(problem_formulation_id)

## Function definition

These helper functions support the execution of ε-NSGAII optimization by setting up convergence metrics, epsilons for each outcome dimension and the actual optimization.

---

### `build_convergence_metrics( )`

Defines how optimization progress will be tracked.
It adds metrics such as ε-progress monitoring and, optionally, logs all non-dominated solutions for later analysis. Useful for assessing convergence quality and reproducibility.

---

### `generate_reference_scenario(...)`

Sets up a reference scenario for the optimization function. Reference values for each Dike Ring are assumed equal.

---

### `generate_epsilons( )`

Automatically generates ε-values tailored to the outcome ranges.
This helps the ε-NSGAII algorithm balance exploration and precision by scaling selection pressure to the variability of each objective.

---

### `run_optimization( )`

Executes the optimization process over the defined decision levers.
Uses parallel processing to efficiently search for Pareto-optimal policies while monitoring convergence with the specified metrics.



In [4]:
def build_convergence_metrics(model):
    """
    Create and return a list of convergence metrics for the optimization process.

    This function sets up logging of the optimization archive and tracks
    convergence progress using the epsilon-progress metric. It uses model
    levers and outcomes to configure the logger.

    Parameters:
        model: An instance of the model with defined `levers` and `outcomes`.

    Returns:
        List containing:
            - ArchiveLogger: Logs decision variables and outcomes to a file.
            - EpsilonProgress: Tracks convergence based on epsilon progress.
    """

    logger = ArchiveLogger(
            r"output",
            decision_varnames=[l.name for l in model.levers],
            outcome_varnames=[o.name for o in model.outcomes],
            base_filename="test_archive.tar.gz",
            )

    metrics = [logger, EpsilonProgress()]

    return metrics

In [5]:
def generate_reference_scenario(discount_rate=1.5,Bmax=190,pfail=0.5,Brate=1):

    """
    Generate a reference scenario for the model with specified default values.

    This function constructs a Scenario object with a fixed set of parameters
    used for reference or baseline evaluations in the model. It defines:
      - Discount rates for 3 dike rings
      - A flood wave shape parameter
      - Bmax, pfail, and Brate for 5 potential adaptation measures

    Parameters:
        discount_rate (float): The discount rate applied to all dike rings.
        Bmax (float): Maximum budget available for dike reinforcement.
        pfail (float): Probability of failure for dike adaptation.
        Brate (float): Budget growth rate for dike adaptation.

    Returns:
        Scenario: A named EMA Workbench Scenario object called 'reference',
                  containing all the specified parameter settings.
    """

    reference_parameters = {
        **{f'discount rate {i}': discount_rate for i in range(3)},
        'A.0_ID flood wave shape': 30,
        **{f'A.{i}_Bmax': Bmax for i in range(1, 6)},
        **{f'A.{i}_pfail': pfail for i in range(1, 6)},
        **{f'A.{i}_Brate': Brate for i in range(1, 6)},
    }

    # Define the scenario
    scenario = Scenario('reference', **reference_parameters)

    return scenario

In [6]:
def generate_epsilons(problem_id, fraction=0.05):
    """
    Generate a list of epsilon values for each outcome based on the outcome ranges.

    This function loads exploratory run results for a given problem, calculates
    the range of each outcome, and returns a list of epsilon values based on a
    specified fraction of those ranges. If the range is too small, a minimum epsilon
    of 1e-5 is used to prevent zero-division or overly strict convergence.

    Parameters:
        problem_id (str or int): Identifier used to locate the results file.
        fraction (float): Fraction of the range to use as the epsilon value for each outcome.

    Returns:
        list of float: Epsilon values corresponding to each outcome.
    """

    _, outcomes = load_results(
        f"../experimental data/pf_{problem_id}_exploratory_runs_levers_as_factors.tar.gz"
    )

    epsilons = []
    for name, values in outcomes.items():
        values = np.asarray(values)
        min_val, max_val = values.min(), values.max()
        range_val = max_val - min_val

        if range_val < 1e-5:
            epsilon = 1e-5  # avoid zero epsilon
        else:
            epsilon = range_val * fraction

        epsilons.append(epsilon)

    return epsilons


In [7]:
def run_optimization(dike_model, nfe, epsilons, convergence_metrics, reference_scenario):
    """
    Run multi-objective optimization on the specified dike model using EMA Workbench.

    This function uses a `MultiprocessingEvaluator` to perform optimization over the
    model's decision levers. The optimization process is guided by epsilon values for
    each outcome and tracked using specified convergence metrics.

    Parameters:
        dike_model: The model instance to be optimized (typically a Model object from EMA Workbench).
        nfe (int): Number of function evaluations to perform during optimization.
        epsilons (list of float): Epsilon values for each outcome, controlling optimization precision.
        convergence_metrics (list): List of convergence metric objects (e.g., ArchiveLogger, EpsilonProgress).
        reference_scenario (Scenario): A baseline scenario used as a reference for convergence metrics.

    Returns:
        tuple: A tuple containing:
            - results: The final set of decision variable and outcome evaluations.
            - convergence: The recorded convergence data collected during optimization.
    """

    print(f"Starting optimization with {nfe} evaluations...")

    with MultiprocessingEvaluator(dike_model) as evaluator:
        results, convergence = evaluator.optimize(
            nfe=nfe,
            searchover="levers",
            epsilons=epsilons,
            convergence=convergence_metrics,
            reference=reference_scenario,
        )

    print(f"Optimization completed.")
    return results, convergence

### A Note on NFE Value Choice

To assess the most appropriate number of iterations, it is standard practice to perform a complete convergence analysis. However, due to coding issues, we were unable to carry this out properly. A detailed explanation of the issues encountered during the convergence analysis is provided at the end of this notebook.

As a result, a value of **50,000** was used for the optimization. This number is generally sufficient for most optimization cases and does not result in an excessively long runtime.


In [8]:
#Setup of optimization algorithm inputs
convergence_metrics = build_convergence_metrics(dike_model)
reference_scenario = generate_reference_scenario()
epsilon = generate_epsilons(problem_formulation_id)
nfe = 50000

#Run evlutionary algorithm
results, convergence = run_optimization(dike_model, nfe, epsilon, convergence_metrics, reference_scenario)

[MainProcess/INFO] results loaded successfully from C:\Users\adell\Main\MBDM\Git_directory\final assignment EPA141\experimental data\pf_3_exploratory_runs_levers_as_factors.tar.gz


Starting optimization with 50000 evaluations...


[MainProcess/INFO] pool started with 16 workers
50338it [1:01:47, 13.58it/s]                                                   
[MainProcess/INFO] optimization completed, found 138 solutions
[MainProcess/INFO] terminating pool


Optimization completed.


In [10]:
#Save the experiments results
results.to_csv('../experimental data/optimization_results_nfe_50000.csv', index=False)

## Convergence Analysis - coding issues

The convergence analysis to assess the appropriate number of iterations for the evolutionary algorithm was not performed till the end due to issues with the coding process.
The following cells will show the setup for the analysis and the issues encountered:


In [11]:
def calculate_metrics(archives, reference_set, reference_scenario):
    """
    Calculate multiple convergence metrics for each archive snapshot during optimization.

    This function evaluates several quality indicators of optimization performance
    using a reference set and scenario. Metrics include hypervolume, generational distance,
    epsilon indicator, inverted generational distance, and spacing. Results are organized
    by number of function evaluations (NFE).

    Parameters:
        archives (dict): A dictionary mapping NFE values to archive DataFrames
                         containing optimization results at each step.
        reference_set (DataFrame): The reference set used for comparison in metric calculations.
        reference_scenario (Scenario): The reference scenario used to construct the problem definition.

    Returns:
        pandas.DataFrame: A DataFrame containing metric scores for each NFE snapshot, sorted by NFE.
    """

    problem = to_problem(dike_model, searchover="levers", reference=reference_scenario)

    hv = HypervolumeMetric(reference_set, problem)
    gd = GenerationalDistanceMetric(reference_set, problem, d=1)
    ei = EpsilonIndicatorMetric(reference_set, problem)
    ig = InvertedGenerationalDistanceMetric(reference_set, problem, d=1)
    sm = SpacingMetric(problem)

    metrics = []
    for nfe, archive in archives.items():
        scores = {
            "generational_distance": gd.calculate(archive),
            "hypervolume": hv.calculate(archive),
            "epsilon_indicator": ei.calculate(archive),
            "inverted_gd": ig.calculate(archive),
            "spacing": sm.calculate(archive),
            "nfe": int(nfe),
        }
        metrics.append(scores)
    metrics = pd.DataFrame.from_dict(metrics)

    # sort metrics by number of function evaluations
    metrics.sort_values(by="nfe", inplace=True)
    return metrics

In [12]:
def plot_metrics(metrics, convergence):
    """
    Plot convergence and performance metrics over the course of optimization.

    This function visualizes how various multi-objective optimization metrics
    evolve as the number of function evaluations (NFE) increases. It plots:

      - Hypervolume
      - Epsilon progress (from convergence data)
      - Generational distance
      - Epsilon indicator
      - Inverted generational distance
      - Spacing

    Parameters:
        metrics (pandas.DataFrame): DataFrame containing metric values for each NFE.
        convergence (pandas.DataFrame): DataFrame with convergence data (e.g., epsilon progress).
    """

    sns.set_style("white")
    fig, axes = plt.subplots(nrows=6, figsize=(8, 12), sharex=True)

    ax1, ax2, ax3, ax4, ax5, ax6 = axes

    ax1.plot(metrics.nfe, metrics.hypervolume)
    ax1.set_ylabel("hypervolume")

    ax2.plot(convergence.nfe, convergence.epsilon_progress)
    ax2.set_ylabel(r'$\epsilon$ progress')

    ax3.plot(metrics.nfe, metrics.generational_distance)
    ax3.set_ylabel("generational distance")

    ax4.plot(metrics.nfe, metrics.epsilon_indicator)
    ax4.set_ylabel("epsilon indicator")

    ax5.plot(metrics.nfe, metrics.inverted_gd)
    ax5.set_ylabel("inverted generational\ndistance")

    ax6.plot(metrics.nfe, metrics.spacing)
    ax6.set_ylabel("spacing")

    ax6.set_xlabel("nfe")

    sns.despine(fig)

In [14]:
# Load optimization results
archives = ArchiveLogger.load_archives(f"./output/test_archive.tar.gz")
reference_set = archives[max(archives.keys())] # this is the final archive

# Calculate metrics and plot
metrics = calculate_metrics(archives, reference_set, reference_scenario)
plot_metrics(metrics, convergence)

plt.show()

EMAError: Parameter names ['0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0', '1_RfR 1', '1_RfR 2', '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0', '3_RfR 1', '3_RfR 2', '4_RfR 0', '4_RfR 1', '4_RfR 2', 'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2', 'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2', 'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2', 'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2', 'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2'] not found in archive

The `HypervolumeMetric` function (called first by `calculate_metrics()`) raises an error due to an issue in the `rebuild_platypus_population()` method.

Specifically, `rebuild_platypus_population()` tries to access parameter values from the archive using `archive.itertuples()`, which returns each row as a `namedtuple`. Namedtuples require attribute-style access (`getattr(row, attr)`), and this only works if attribute names are valid Python identifiers.

However, many of the `problem.parameter_names` contain spaces, dots, or other invalid characters (e.g., `'0_RfR 0'`, `'A.1_DikeIncrease 1'`). When Pandas uses `itertuples()`, these characters are automatically sanitized (e.g., spaces become underscores), or in some cases, replaced with `_0`, `_1`, etc., if the name cannot be made valid. This mismatch between original parameter names and the sanitized attribute names causes `getattr(row, attr)` to fail with an `AttributeError`.

As a result, the error message indicates that the parameter names could not be found in the archive — not because they don’t exist, but because the attribute-style lookup cannot match the original names due to the sanitization performed by `itertuples()`.

This can be confirmed easily by printing the attributes of each row returned by `itertuples()`:



In [15]:
for row in reference_set.itertuples():
    print(dir(row))

['EWS_DaysToThreat', 'Index', '_1', '_10', '_11', '_12', '_13', '_14', '_15', '_17', '_18', '_19', '_2', '_20', '_21', '_22', '_23', '_24', '_25', '_26', '_27', '_28', '_29', '_3', '_30', '_31', '_32', '_33', '_34', '_35', '_36', '_37', '_38', '_39', '_4', '_40', '_41', '_42', '_43', '_5', '_6', '_7', '_8', '_9', '__add__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getnewargs__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__match_args__', '__module__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '_asdict', '_field_defaults', '_fields', '_make', '_replace', 'count', 'index']
['EWS_DaysToThreat', 'Index', '_1', '_10', '_11', '_12', '_13', '_14', '_15', '_17', '_18', '_19', '_2', '_20', '_